In [ ]:
import pandas as pd
import dash
from dash import html
import plotly.graph_objects as go
from dash import dash_table
import plotly.express as px
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input,Output
import numpy as np

## Web App Definition

In [ ]:
app = dash.Dash()

## Perform transformations on main data set

In [ ]:
df = pd.read_csv('tely10.csv')
df = df.drop(['index'],axis=1)

In [ ]:
gun_times = df['gun_time'].squeeze()
chip_times = df['chip_time'].squeeze()

In [ ]:
def fix_hour(value):
    
    if(len(value) < 7):
        
        value = '00:' + value
        return value
        
    else:
        
        return value

In [ ]:
new_gun_times = gun_times.apply(fix_hour)
new_chip_times = chip_times.apply(fix_hour)

In [ ]:
df['new_gun_times'] = new_gun_times
df['new_chip_times'] = new_chip_times

df['new_gun_times'] = pd.to_datetime(df['new_gun_times'])
df['new_chip_times'] = pd.to_datetime(df['new_chip_times'])

df = df.drop(['gun_time'],axis=1)
df = df.drop(['chip_time'],axis=1)
df = df.rename(columns={"new_gun_times": "gun_time", "new_chip_times": "chip_time"})

In [ ]:
df['diff_seconds'] = (df.gun_time - df.chip_time).dt.seconds

In [ ]:
def assign_gender(gender):
    
    if(gender == 'LM'):
        
        gender = 'Men'
        return gender
        
    elif(gender == 'LF'):
        
        gender = 'Women'
        return gender
    
    else:
        
        return

In [ ]:
df['gender'] = df.division.str[:2]

df['gender'] = df['gender'].apply(assign_gender)

## Code for first plot

In [ ]:
df_plot1 = df.groupby('year').count()
df_plot1 = df_plot1.reset_index()
df_plot1 = df_plot1[['year', 'overall_place']]
df_plot1 = df_plot1.rename(columns={"year": "Year", "overall_place": "Total Participants"})

df_plot1_m = df[df['gender'] == 'Men'].groupby('year').count()
df_plot1_m = df_plot1_m.reset_index()
df_plot1_m = df_plot1_m[['year', 'overall_place']]
df_plot1_m = df_plot1_m.rename(columns={"year": "Year", "overall_place": "Total Participants"})

df_plot1_w = df[df['gender'] == 'Women'].groupby('year').count()
df_plot1_w = df_plot1_w.reset_index()
df_plot1_w = df_plot1_w[['year', 'overall_place']]
df_plot1_w = df_plot1_w.rename(columns={"year": "Year", "overall_place": "Total Participants"})

In [ ]:
def line_chart(df, df1, df2):
    
    fig = go.Figure()
    
    
    fig.add_trace(go.Scatter(x = df['Year'], y = df['Total Participants'],\
                             mode = 'lines+markers', name = 'Total Participants'))
    
    fig.add_trace(go.Scatter(x = df1['Year'], y = df1['Total Participants'],\
                             mode = 'lines+markers', name = 'Men'))
    
    fig.add_trace(go.Scatter(x = df2['Year'], y = df2['Total Participants'],\
                             mode = 'lines+markers', name = 'Women'))
    
    
    fig.update_layout(title='Number of Participants per Year in Tely 10 Race',
                      title_x = 0.5,
                       xaxis_title='Years',
                       yaxis_title='Number of Participants',
                      font=dict(
                        family="Arial",
                        size=18,
                        color="black"
                        )
                       )
    
    fig.add_annotation(x = 2017, y = 4354,
                      text = 'Highest number of participants in the last 15 years (4354)',
                      showarrow = True,
                      arrowhead = 1,
                      font=dict(
                        family="Arial",
                        size=15,
                        color="black"
                        )
                       )
    
    fig.add_annotation(x = 2017, y = 2680,
                      text = 'Highest number of women participants in the last 15 years (2680)',
                      showarrow = True,
                      arrowhead = 1,
                      ay = -20,
                      font=dict(
                        family="Arial",
                        size=15,
                        color="black"
                        )
                       )
    
    fig.add_annotation(x = 2016, y = 1712,
                      text = 'Highest number of men participants in the last 15 years (1712)',
                      showarrow = True,
                      arrowhead = 1,
                      ay = 40,
                      font=dict(
                        family="Arial",
                        size=15,
                        color="black"
                        )
                       )
    
    return fig

## Code for second plot

In [ ]:
df_plot2 = df[['year', 'diff_seconds']]

## Code for third plot

In [ ]:
df_plot3 = df[['year', 'overall_place', 'name', 'chip_time']]

df_plot3 = df_plot3[df_plot3['overall_place'] <= 10.0].reset_index()

## Code for fourth plot

In [ ]:
df_plot4 = df[['year', 'overall_place', 'gender_place', 'chip_time', 'division', 'gender']]

df_plot4 = df_plot4.dropna()

df_plot4_men = df_plot4[(df_plot4.gender_place == 1.0) & (df_plot4.gender == 'Men')].reset_index()

df_plot4_men = df_plot4_men.drop(['index'],axis=1)

df_plot4_women = df_plot4[(df_plot4.gender_place == 1.0) & (df_plot4.gender == 'Women')].reset_index()

df_plot4_women = df_plot4_women.drop(['index'],axis=1)

In [ ]:
def line_chart4_mw(df):
    
    fig = go.Figure()
    
    df_plot4_men = df[(df.gender_place == 1.0) & (df.gender == 'Men')]
    df_plot4_women = df[(df.gender_place == 1.0) & (df.gender == 'Women')]
    
    fig.add_trace(go.Scatter(x = df_plot4_men['year'], y = df_plot4_men['chip_time'],\
                             mode = 'lines+markers', name = 'Men'))
    
    fig.add_trace(go.Scatter(x = df_plot4_women['year'], y = df_plot4_women['chip_time'],\
                             mode = 'lines+markers', name = 'Women'))
    
    fig.update_layout(title="Best Time per Year For Both Men and Women",
                      title_x = 0.5,
                       xaxis_title='Years',
                       yaxis_title='Time',
                      font=dict(
                        family="Arial",
                        size=18,
                        color="black"
                        )
                       )
    
    fig.add_annotation(x = 2019, y = df_plot4_women['chip_time'].min(),
                      text = 'Lowest female time recorded in the last 15 years (54:24)',
                      showarrow = True,
                      arrowhead = 1,
                      ay = -120,
                      font=dict(
                        family="Arial",
                        size=15,
                        color="black"
                        )
                       )
    
    fig.add_annotation(x = 2012, y = df_plot4_men['chip_time'].min(),
                      text = 'Lowest male time recorded in the last 15 years (48:09)',
                      showarrow = True,
                      arrowhead = 1,
                      ay = -80,
                      font=dict(
                        family="Arial",
                        size=15,
                        color="black"
                        )
                       )
    
    return fig

## Dashboard Web App Building

In [ ]:
app.layout = html.Div(id = 'parent', children = [
    
    html.Div(id = 'H1', children = 'Tely 10 Race Dashboard', style = {'color':'black', 'fontSize':'50px',\
                                                                      'fontFamily':'Arial', 'text-align': 'center',\
                                                                     'marginTop':40,'marginBottom':40,\
                                                                           'text-decoration':'underline'}),
    
    html.Br(),
    
    html.H1(id = 'H1_p1', children = 'Growth of Tely 10 Race Over The Years', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),
    
    html.Div(id = 'line_div_1', children = [dcc.Graph(id = 'line_plot_1', figure = line_chart(df_plot1, df_plot1_m, df_plot1_w))],style = {'width':'100%','display':'inline-block'}),
    
    html.Br(),
    
    html.H1(id = 'H1_p2', children = 'Accuracy of Time Records Between Gun Time and Chip Time', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),
    
    html.Div([
                dcc.Dropdown( id = 'year-dropdown', options = [{'label':i, 'value':i} for i in np.append(['All'],df_plot2['year'].unique()) ],
                 value = 'All', placeholder = 'Select the year', style = {'width':'100%', 'display':'inline-block'}),
        
                dcc.Graph(id="graph"),
            ]),
    
    html.Br(),
    html.Br(),
    
    html.H1(id = 'H1_p3', children = 'Times Recorded for Top 10 Participants', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),
    
    html.Div([
                dcc.Dropdown( id = 'year-dropdown_2', options = [{'label':i, 'value':i} for i in np.append(['All'],df_plot3['year'].unique()) ],
                 value = 'All', placeholder = 'Select the year', style = {'width':'100%', 'display':'inline-block'}),
        
                dcc.Graph(id="graph_2"),
            ]),
    
    html.Br(),
    html.Br(),
    
    html.H1(id = 'H1_p4', children = """Changes in Winner's Times""", style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),
    
    html.Div(id = 'line_div_4_w', children = [dcc.Graph(id = 'line_plot_4_w', figure = line_chart4_mw(df_plot4))],style = {'width':'100%','display':'inline-block'})


])

## Callback funtion for second plot

In [ ]:
@app.callback(Output("graph", "figure"), 
             Input("year-dropdown", "value"))
              
def display_hist(year):
   
    if year == 'All':
        filtered_df = df_plot2
    else :
        filtered_df = df_plot2[df_plot2['year'] == int(year)]

    fig = go.Figure(data=[go.Histogram(x=filtered_df['diff_seconds'], xbins_size = 5)])
    
    if(year == 'All'):
        
        year = year + " years"
        
    fig.update_traces(marker_color="#69EE5F")

    fig.update_layout(title='Difference in Time Between Gun Time and Chip Time in {}'.format(year),
                  title_x = 0.5,
                   xaxis_title= 'Gun Time - Chip Time (s)',
                  font=dict(
                    family="Arial",
                    size=18,
                    color="black"
                    )
                   )

    fig.update(layout_xaxis_range = [0,360])

    fig.show()

    return fig

## Callback function for plot 3

In [ ]:
@app.callback(Output("graph_2", "figure"), 
             Input("year-dropdown_2", "value"))
              
def display_bar(year):
   
    if year == 'All':
        #filtered_df = df_plot3.sort_values(by = 'chip_time', ascending = True)
        #filtered_df = filtered_df.head(10)
        filtered_df = df_plot3.sort_values(by = 'chip_time').drop_duplicates(subset = ['name']).head(10)
        
    else :
        filtered_df = df_plot3[df_plot3['year'] == int(year)]

    fig = go.Figure(data=[go.Bar(x=filtered_df['name'], y = filtered_df['chip_time'])])

    fig.update_layout(title='Times Recorded for Top 10 Participants',
                  title_x = 0.5,
                   xaxis_title= 'Names of Participants',
                   yaxis_title= 'Time Recorded',
                   height = 600,
                  font=dict(
                    family="Arial",
                    size=18,
                    color="black"
                    ),
                    xaxis_tickangle = -90
                   )
    fig.show()

    return fig

In [ ]:
if __name__ == '__main__': 
    app.run_server()